In [1]:
import numpy as np
import torch

from leveraging_geometry_for_shape_estimation.models.represent_3d_object.extract_points_from_models import find_edge_points,load_points_normals,sample_points_from_lines,repair_mesh,plot_points,different_normals_around_points,vis_representative_normals
from leveraging_geometry_for_shape_estimation.models.represent_3d_object.moller_trumbore import inside_mesh, moller_trumbore,normalize


from pytorch3d.io import load_objs_as_meshes, save_obj, load_obj
import pytorch3d
import k3d


In [2]:
path = '/scratch/fml35/datasets/shapenet_v2/ShapeNetRenamed/representation_points_and_normals/exp_02/points_and_normals/bathtub_4a6ed9c61b029d15904e03fa15bd73ec.npz'
# model_path = '/scratch2/fml35/datasets/shapenet_v2/ShapeNetRenamed/model/cabinet/23aeb8fdc0d1ed4c4da5feafe6f1c8fc/model_normalized.obj'

model_path = '/scratch2/fml35/datasets/shapenet_v2/ShapeNetRenamed/model/bathtub/4a6ed9c61b029d15904e03fa15bd73ec/model_normalized.obj'
point_norm = np.load(path)
rotation_path = '/scratch2/fml35/experiments/leveraging_geometry_for_shape/exp_117_scannet_models/models/rotations/R_T_torch.npz'
device = torch.device("cuda:0")

# /scratch2/fml35/experiments/leveraging_geometry_for_shape/exp_117_scannet_models/models/representation_points_and_normals/exp_01/vis/cabinet_23aeb8fdc0d1ed4c4da5feafe6f1c8fc.ply

print(point_norm['points'].shape)
print(point_norm['normals'].shape)

plot_points(point_norm['points'],point_size=0.03)

(300, 3)
(300, 3, 3)


Output()

In [3]:
R_and_T = np.load(rotation_path)
R_mesh = torch.from_numpy(R_and_T["R"]).to(device)
T_mesh = torch.from_numpy(R_and_T["T"]).to(device)
R_mesh = torch.inverse(R_mesh)

elev = [0.0,15.0,30.0,45.0]
azim = [0.0,22.5,45.0,67.5,90.0,112.5,135.0,157.5,180.0,202.5,225.0,247.5,270.0,292.5,315.0,337.5]

gamma = 0.001

print('R_mesh',R_mesh.shape)
print('T_mesh',T_mesh.shape)

vertices,faces,textures = load_obj(model_path ,load_textures=False,device=device)
faces = faces[0].long()

mask_dict = {}

points_sparse = torch.from_numpy(point_norm['points']).to(device)
normals_sparse = torch.from_numpy(point_norm['normals']).to(device)

query_point_index = 100

total_index = 0
for i in range(16):
    for j in range(4):
        v_mesh = torch.transpose(torch.matmul(R_mesh[total_index,:,:],torch.transpose(vertices,0,1)),0,1) + T_mesh[total_index,:]


        points = torch.transpose(torch.matmul(R_mesh[total_index,:,:],torch.transpose(points_sparse,0,1)),0,1) + T_mesh[total_index,:]
        # print('torch.mean(normals_sparse,dim=2)',torch.mean(normals_sparse,dim=2)[:2])
        # print('torch.mean(normals_sparse,dim=1)',torch.mean(normals_sparse,dim=1)[:2])
        # points = points.unsqueeze(1)
        # points = points.repeat(1,2,1)

        ray_o = points * 0
        ray_d = normalize(points)
        faces_muller = v_mesh[faces]

        # print('ray_o',ray_o.shape)
        # print('ray_d',ray_d.shape)
        # print('faces',faces.shape)

        u, v, t = moller_trumbore(ray_o, ray_d, faces_muller)  # (n_rays, n_faces, 3)
        # print('u',u[:2,:20])
        # print('u.v.t', u.shape, v.shape,t.shape)
        # points_outward = points + 0.03*torch.mean(normals_sparse,dim=2)
        # plot_points(points_outward.cpu().numpy(),point_size=0.03)


        intersection = ((t >= 0.0) * (u >= 0.0) * (v >= 0.0) * ((u + v) <= 1.0)).bool()  # (n_rays, n_faces)
        intersects = torch.nonzero(intersection)





        involved_points = v_mesh[faces[intersects[:, 1]]]
        # print('involved_points',involved_points.shape)
        center_intersects = torch.mean(involved_points, dim=1)  # (n_faces, 3)
        # print('center_intersects',center_intersects.shape)
        # print(intersects[:10])
        # print('intersection', intersection)
        # print('intersects', intersects.shape)
        points_sparse_same_as_intersects = points[intersects[:, 0]]

        # print('points_sparse_same_as_intersects',points_sparse_same_as_intersects[:20])

        distances_to_orig_face_centers = torch.norm(center_intersects, dim=1)
        distances_to_orig_points = torch.norm(points_sparse_same_as_intersects, dim=1)

        smaller_than_distance_face = distances_to_orig_face_centers > distances_to_orig_points - gamma

        max_len = torch.max(torch.bincount(intersects[:, 0]))
        point_dict_counter = {}
        smaller_than_distance_reshaped = torch.ones((points_sparse.shape[0],max_len))


        for k in range(smaller_than_distance_face.shape[0]):
            index = int(intersects[k, 0].item())
            # if index == query_point_index:

            # print('smaller_than_distance_reshaped',smaller_than_distance_reshaped[query_point_index])
            
            if index not in point_dict_counter:
                smaller_than_distance_reshaped[index, 0] = smaller_than_distance_face[k]
                point_dict_counter[index] = 1
            else:
                # print('yes increase')
                smaller_than_distance_reshaped[index, point_dict_counter[index]] = smaller_than_distance_face[k]
                point_dict_counter[index] += 1
            # print('point_dict_counter[index]',point_dict_counter[index])
        
        # print('smaller_than_distance_reshaped',smaller_than_distance_reshaped[query_point_index])
        mask = torch.all(smaller_than_distance_reshaped,dim=1)

        # print('smaller_than_distance_face',smaller_than_distance_face[intersects[:, 0] == query_point_index])
        # print('mask[query_point_index]',mask[query_point_index])

        print('mask',torch.sum(mask))
        # print('smaller_than_distance_face',smaller_than_distance_face[:20])
        # print('distances',distances.shape)
        # print('distances',distances[:20])
        # print('intersetion',intersects[:20])

        # print(df)
        # inside = (intersection.count_nonzero(dim=-1) % 2).bool()

        # print('intersection.shape', intersection.shape)
        # print('inside.shape', inside.shape)

        # plot = k3d.plot()
        # # plot += k3d.points(points.cpu().numpy(), point_size=0.006,color=0xff0000 ,shader="flat")
        # plot += k3d.points(points.cpu().numpy()[query_point_index], point_size=0.02,color=0xff ,shader="flat")
        # plot += k3d.points(center_intersects[intersects[:,0] == query_point_index,:].cpu().numpy(), point_size=0.006,color=0xf0f0 ,shader="flat")
        # plot += k3d.points(points[mask].cpu().numpy(), point_size=0.02,color=0xffff ,shader="flat")
        # plot += k3d.mesh(v_mesh.cpu().numpy(), faces[0].cpu().numpy(), textures=None, shader="flat")
        # plot.display()
        
        # # take 3d points, move outward by small amount by average of their normals
        # # for each point check if ray from 0 to point intersects a mesh surface


        elev_index = total_index // len(azim)
        # NOTE: complicated formula because of old convention
        azim_index = (len(azim) - total_index % len(azim)) % len(azim)
        azim_index = total_index % len(azim)
        print('elev',elev)
        print('azim',azim)
        print('elev_index',elev_index)
        elev_current = str(int(elev[elev_index])).zfill(3)
        azim_current = str(np.round(azim[azim_index],1)).zfill(3)

        name = 'elev_{}_azim_{}.npy'.format(elev_current,azim_current)
        mask_dict[name] = mask.cpu().numpy()

        total_index += 1
        

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.